In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [10]:
df = pd.read_csv('/home/saumyaryan/Documents/PR/PR_201/Files/creditcard.csv')

# Select features (excluding label 'Fraud')
df = df[df['Class'] == 1]
features = df.drop(columns=['Class']).values
labels = df['Class'].values  # Use labels for evaluation, not for GAN training

# Normalize the data for better GAN performance
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Convert to PyTorch tensors
real_data = torch.tensor(features_scaled, dtype=torch.float32)

In [20]:
# Generator Model
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Discriminator Model
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [24]:
input_dim = real_data.shape[1]  # Number of features in the dataset
z_dim = 20  # Latent space size for generator

generator = Generator(z_dim, input_dim)
discriminator = Discriminator(input_dim)


criterion = nn.BCELoss()
lr = 0.0002

optimizer_G = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0001)  # Slower learning for D



In [26]:
batch_size = 64
num_epochs = 10000

for epoch in range(num_epochs):
    # Select a random batch from real data
    idx = torch.randint(0, real_data.size(0), (batch_size,))
    real_batch = real_data[idx]

    noise = torch.randn_like(real_batch) * 0.05
    real_batch_noisy = real_batch + noise


    # Generate fake data
    z = torch.randn(batch_size, z_dim)
    fake_data = generator(z)

    # Labels for real (1) and fake (0)
    real_labels = torch.full((batch_size, 1), 0.9)  # Instead of 1
    fake_labels = torch.full((batch_size, 1), 0.1)  # Instead of 0


    # Train Discriminator
    real_loss = criterion(discriminator(real_batch_noisy), real_labels)
    fake_loss = criterion(discriminator(fake_data.detach()), fake_labels)
    d_loss = real_loss + fake_loss

    optimizer_D.zero_grad()
    d_loss.backward()
    optimizer_D.step()

    # Train Generator
    g_loss = criterion(discriminator(fake_data), real_labels)

    optimizer_G.zero_grad()
    g_loss.backward()
    optimizer_G.step()

    if epoch % 1000 == 0:
        print(f"Epoch {epoch}/{num_epochs}, D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")


Epoch 0/10000, D Loss: 1.5405288934707642, G Loss: 0.6990412473678589
Epoch 1000/10000, D Loss: 1.282390832901001, G Loss: 0.7915784120559692
Epoch 2000/10000, D Loss: 1.3745840787887573, G Loss: 0.7155547142028809
Epoch 3000/10000, D Loss: 1.3551385402679443, G Loss: 0.7446081638336182
Epoch 4000/10000, D Loss: 1.3326581716537476, G Loss: 0.7493083477020264
Epoch 5000/10000, D Loss: 1.3552424907684326, G Loss: 0.781499445438385
Epoch 6000/10000, D Loss: 1.3720362186431885, G Loss: 0.6782967448234558
Epoch 7000/10000, D Loss: 1.4089906215667725, G Loss: 0.7071007490158081
Epoch 8000/10000, D Loss: 1.4820038080215454, G Loss: 0.6851615309715271
Epoch 9000/10000, D Loss: 1.3936870098114014, G Loss: 0.7175788879394531
